In [1]:
import pandas as pd
import numpy as np
import re
import json
import matplotlib.pyplot as plt
import glob, os, csv

import ast
from collections import Counter
from operator import add

from sklearn.model_selection import train_test_split
import math
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction import DictVectorizer
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import VotingClassifier, StackingClassifier, AdaBoostClassifier
import seaborn as sns
from sklearn.metrics import classification_report
# from xgboost import XGBClassifier
pd.options.mode.chained_assignment = None

In [2]:
import tensorflow as tf
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split


from transformers import AlbertModel, AlbertTokenizer, AlbertForSequenceClassification, AlbertConfig
from transformers import AdamW

from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# Preprocessing

In [4]:
## Categories ##
# Market: Drug, gun, 
# Counterfeit: counterfeit credit cards, money, ID
# Services: Hosting service, forum, email, pastebin, file-sharing
# Security: Security-related information, tutorials or services, leaked data
# Porn: Hosting pornographic material
# Cryptocurrency
# NoAccess: Login, Down, Empty
# Other: Cannot be classified in any other category (e.g. personal blog)

In [5]:
# Dictionary of conversion of categories
conversion_dict = {
    'Art': 'Other',
    'Casino': 'Services',
    'Counterfeit Credit-Cards': 'Counterfeit',
    'Counterfeit Money': 'Counterfeit',
    'Counterfeit Personal-Identification': 'Counterfeit',
    'Cryptocurrency': 'Cryptocurrency',
    'Cryptolocker': 'Security',
    'Down': 'NoAccess',
    'Drugs': 'Market',
    'Empty': 'NoAccess',
    'Forum': 'Services',
    'Hacking': 'Security',
    'Hosting': 'Services',
    'Leaked-Data': 'Security',
    'Library': 'Other',
    'Locked': 'NoAccess',
    'Marketplace': 'Market',
#     'Onion Directory/Wiki': ['directory', 'dir', 'wiki'],
    'Personal': 'Other',
    'Politics': 'Other',
    'Porno': 'Porn',
    'Religion': 'Other',
    'Services': 'Services',
    'Social-Network': 'Services',
    'Violence': 'Market'
}

In [6]:
# Read input data
df_combined = pd.read_csv('../data/model_training_dataset.csv')

# convert the string representation of a list into a list
df_combined['url'] = df_combined['url'].apply(ast.literal_eval)
df_combined['body_text'] = df_combined['body_text'].apply(ast.literal_eval)
df_combined['body_token'] = df_combined['body_token'].apply(ast.literal_eval)
df_combined['class'] = df_combined['class'].apply(lambda x: x.strip())
df_combined

In [9]:
df_combined_eng = df_combined[df_combined['language'] == 'en']
df_combined_eng.reset_index(inplace=True)
df_combined_eng

,url,title,body_text,body_token,language,class
0,[wwjewp6ca4rkudyj.onion],\n Frënn vun der Ënn - Index\n,"[Frënn vun der Ënn - Index, Skip to main conte...","[frënn, vun, der, ënn, index, skip, content, e...",en,Services
1,[u76xgym22s4adf55.onion],PayPal Master: Home,"[PayPal Master: Home, Contact us at: ppmaster@...","[master, home, contact, ppmaster, deepdarkmail...",en,Counterfeit
2,[2hftxvyft7dl3fk2.onion],Umbrella Escrow,"[Umbrella Escrow, Start new escrow, View exist...","[umbrella, escrow, escrow, view, escrow, instr...",en,Cryptocurrency
3,[elherbotsiddarol.onion],ElHerbolario,"[ElHerbolario, My Account, Blog, PGP, Sitemap,...","[elherbolario, account, blog, pgp, sitemap, nb...",en,Market
4,[yermrrzeg4fusqx5.onion],anonymous OnionMail Server,"[anonymous OnionMail Server, anonymous OnionMa...","[onionmail, server, onionmail, serveronion, ma...",en,Services
...,...,...,...,...,...,...
1981,[5wkkhgqtity7bhdf.onion],walletGenerator - Universal Paper wallet gener...,[walletGenerator - Universal Paper wallet gene...,"[walletgener, univers, paper, wallet, gener, b...",en,Cryptocurrency
1982,[ctzzqqimlfamyhrc.onion],| TheYOSH.nl,"[| TheYOSH.nl, Skip to main content, User acco...","[theyosh, nl, skip, content, user, account, me...",en,Other
1983,[grrmailb3fxpjbwm.onion],✉ Guerrilla Mail on Tor,"[✉ Guerrilla Mail on Tor, TorGuerrillaMail - D...","[guerrilla, mail, tor, dispos, mail, address, ...",en,Services
1984,[greenegbqkyk3ois.onion],✔ Legit Carding Services ♛ Money Transfer Worl...,[✔ Legit Carding Services ♛ Money Transfer Wor...,"[card, servic, money, transfer, worldwid, webs...",en,Counterfeit


In [10]:
# Count number of clusters in each category
df_combined_eng_count = df_combined_eng[['title', 'class']].groupby(['class']).count()
df_combined_eng_count = df_combined_eng_count.sort_values(by=['title'], ascending=False)
df_combined_eng_count.reset_index(inplace=True)
df_combined_eng_count

,class,title
0,Services,574
1,Counterfeit,530
2,NoAccess,212
3,Market,208
4,Cryptocurrency,128
5,Other,125
6,Security,80
7,Porn,70


In [11]:
# plt.figure(figsize=(20,10))
# plt.bar(df_combined_eng_count['class'], df_combined_eng_count['title'])
# plt.xlabel("Category")
# plt.ylabel("Number of Clusters")
# plt.title("Number of Clusters in Each Category")
# plt.show()

In [12]:
# plt.figure(figsize=(20,20))
# plt.pie(df_combined_eng_count['title'], labels=df_combined_eng_count['class'], autopct='%1.1f%%')
# plt.title("Propotion of Clusters in Each Category")
# plt.show()

In [13]:
# df_combined.to_csv('../data/universe_sample_verified.csv')

In [14]:
# List of labels
labels_list = df_combined_eng['class'].values
# Encode L as integers
le = preprocessing.LabelEncoder()
le.fit(labels_list)
labels = le.transform(labels_list)
# Get the number of labels
set(labels)

{0, 1, 2, 3, 4, 5, 6, 7}

In [15]:
# List of original body texts
texts = df_combined_eng['body_text'].values
# Explore the lengths of body texts
texts_len = [len(txt) for txt in texts]
print(max(texts_len))
print(min(texts_len))
print(sum(texts_len)/len(texts_len))

49921
1
541.9889224572004


In [16]:
# List of body texts without duplicates
texts_set = []
for txt in texts:
    txt = set(txt)
    texts_set.append(txt)

# Print the length
texts_set_len = [len(txt) for txt in texts_set]
print(max(texts_set_len))
print(min(texts_set_len))
print(sum(texts_set_len)/len(texts_set_len))

7207
1
120.21752265861028


In [17]:
texts_string = ['[CLS]' + (' [SEP] ').join(txt) + ' [SEP] ' for txt in texts_set]
texts_string[0]

'[CLS]Organization [SEP] Partners [SEP] Services [SEP] Join Us [SEP] Skip to main content [SEP] Abuse [SEP] Worldmap [SEP] Support [SEP] Donation history [SEP] Become a Member [SEP] Mirrors [SEP] About [SEP] Learn more about Tor [SEP] Mailing List [SEP] Frënn vun der Ënn - Index [SEP] Frënn vun der Ënn A.S.B.L. [SEP] @FrennVunDerEnn [SEP] Wiki [SEP] Adopt a Bridge [SEP] Twitter [SEP] Privacy is the right to a free mind! [SEP] Statistics [SEP] I support FVDE because even small organizations can contribute to protect fundamental civil rights. This noble goal can be pursued by making use of technology and putting knowledge to the service of users who want to maintain their privacy. [SEP] Georges, Teacher [SEP] Traffic [SEP] General [SEP] Enn.lu [SEP] CC-BY-NC-SA [SEP] News [SEP] Luxembourg based non-profit organization defending civil rights on the internet. [SEP] Frënn vun der Enn a.s.b.l. (R.C.S. Luxembourg F 9.478) [SEP] About Ënnstatus [SEP] Edward J. Snowden [SEP] Contact [SEP] By bo

In [18]:
tokenizer = AlbertTokenizer.from_pretrained('albert-large-v2', do_lower_case=True)

In [19]:
tokenized_texts = [tokenizer.tokenize(txt) for txt in texts_string]
tokenized_texts[0]

['[CLS]',
 '▁organization',
 '[SEP]',
 '▁partners',
 '[SEP]',
 '▁services',
 '[SEP]',
 '▁join',
 '▁us',
 '[SEP]',
 '▁skip',
 '▁to',
 '▁main',
 '▁content',
 '[SEP]',
 '▁abuse',
 '[SEP]',
 '▁world',
 'map',
 '[SEP]',
 '▁support',
 '[SEP]',
 '▁donation',
 '▁history',
 '[SEP]',
 '▁become',
 '▁a',
 '▁member',
 '[SEP]',
 '▁mirrors',
 '[SEP]',
 '▁about',
 '[SEP]',
 '▁learn',
 '▁more',
 '▁about',
 '▁to',
 'r',
 '[SEP]',
 '▁mail',
 'ing',
 '▁list',
 '[SEP]',
 '▁fren',
 'n',
 '▁v',
 'un',
 '▁der',
 '▁en',
 'n',
 '▁',
 '-',
 '▁index',
 '[SEP]',
 '▁fren',
 'n',
 '▁v',
 'un',
 '▁der',
 '▁en',
 'n',
 '▁a',
 '.',
 's',
 '.',
 'b',
 '.',
 'l',
 '.',
 '[SEP]',
 '▁',
 '@',
 'f',
 'ren',
 'nv',
 'under',
 'en',
 'n',
 '[SEP]',
 '▁',
 'wiki',
 '[SEP]',
 '▁adopt',
 '▁a',
 '▁bridge',
 '[SEP]',
 '▁twitter',
 '[SEP]',
 '▁privacy',
 '▁is',
 '▁the',
 '▁right',
 '▁to',
 '▁a',
 '▁free',
 '▁mind',
 '!',
 '[SEP]',
 '▁statistics',
 '[SEP]',
 '▁i',
 '▁support',
 '▁f',
 'v',
 'de',
 '▁because',
 '▁even',
 '▁small',
 '

In [20]:
# Explore the lengths of body texts
tokens_len = [len(txt) for txt in tokenized_texts]
print(max(tokens_len))
print(min(tokens_len))
print(sum(tokens_len)/len(tokens_len))

251403
3
2196.808157099698


In [21]:
# Set the maximum sequence length.
# MAX_LEN = 128
MAX_LEN = 512

In [22]:
# Use the tokenizer to convert the tokens to their index numbers in the vocabulary
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
# Pad our input tokens
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
input_ids

array([[    2,  1165,     3, ...,     0,     0,     0],
       [    2,    95,    50, ...,   878, 21012,   676],
       [    2, 13447,    13, ...,     0,     0,     0],
       ...,
       [    2,  7582,  2109, ...,     0,     0,     0],
       [    2,  1302,    91, ...,    10,  6664,  2032],
       [    2,    13,     8, ...,     0,     0,     0]])

In [23]:
input_ids.shape

(1986, 512)

In [24]:
# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

In [25]:
# Use train_test_split to split our data into train and validation sets for training
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, random_state=2018, test_size=0.1)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids, random_state=2018, test_size=0.1)

In [26]:
# Convert all of our data into torch tensors, the required datatype for our model
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

In [27]:
# Select a batch size for training. For fine-tuning, the authors recommend a batch size of 32, 48, or 128. We will use 32 here to avoid memory issues.
# batch_size = 32
batch_size = 128
# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop, 
# with an iterator the entire dataset does not need to be loaded into memory
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [28]:
# Define model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load AlbertForSequenceClassification, the pretrained ALBERT model with a single linear classification layer on top. 
model = AlbertForSequenceClassification.from_pretrained("albert-large-v2", num_labels=8)
model.to(device)

Some weights of the model checkpoint at albert-large-v2 were not used when initializing AlbertForSequenceClassification: ['predictions.bias', 'predictions.LayerNorm.weight', 'predictions.LayerNorm.bias', 'predictions.dense.weight', 'predictions.dense.bias', 'predictions.decoder.weight', 'predictions.decoder.bias']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-large-v2 and are newly initialized: ['classifier.weight', 'classifier.bias']
You s

AlbertForSequenceClassification(
  (albert): AlbertModel(
    (embeddings): AlbertEmbeddings(
      (word_embeddings): Embedding(30000, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
    (encoder): AlbertTransformer(
      (embedding_hidden_mapping_in): Linear(in_features=128, out_features=1024, bias=True)
      (albert_layer_groups): ModuleList(
        (0): AlbertLayerGroup(
          (albert_layers): ModuleList(
            (0): AlbertLayer(
              (full_layer_layer_norm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
              (attention): AlbertAttention(
                (query): Linear(in_features=1024, out_features=1024, bias=True)
                (key): Linear(in_features=1024, out_features=1024, bias=True)
                (value): Linear(in_features=1024, out_featur

In [29]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

In [30]:
# This variable contains all of the hyperparemeter information our training loop needs
optimizer = AdamW(optimizer_grouped_parameters, lr=2e-5)

In [31]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [32]:
# Store our loss and accuracy for plotting
train_loss_set = []

# Number of training epochs (authors recommend between 2 and 4)
epochs = 4

# trange is a tqdm wrapper around the normal python range
for _ in trange(epochs, desc="Epoch"):
    # Training
    # Set our model to training mode (as opposed to evaluation mode)
    model.train()

    # Tracking variables
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0

    # Train the data for one epoch
    for step, batch in enumerate(train_dataloader):
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        # Clear out the gradients (by default they accumulate)
        optimizer.zero_grad()
        # Forward pass
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        loss = outputs[0]
        logits = outputs[1]
        train_loss_set.append(loss.item())    
        # Backward pass
        loss.backward()
        # Update parameters and take a step using the computed gradient
        optimizer.step()


        # Update tracking variables
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1

    print("Train loss: {}".format(tr_loss/nb_tr_steps))


    # Validation

    # Put model in evaluation mode to evaluate loss on the validation set
    model.eval()

    # Tracking variables 
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        # Telling the model not to compute or store gradients, saving memory and speeding up validation
        with torch.no_grad():
            # Forward pass, calculate logit predictions
            output = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
            logits = output[0]

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        tmp_eval_accuracy = flat_accuracy(logits, label_ids)

        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))

Epoch:   0%|          | 0/4 [00:12<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
plt.figure(figsize=(15,8))
plt.title("Training loss")
plt.xlabel("Batch")
plt.ylabel("Loss")
plt.plot(train_loss_set)
plt.show()